### Likelihood-Free Parameter Inference on the MA2 Model

This notebook illustrates neural network-based (henceforth referred to as ANN) inference and approximate Bayesian computation.

The ANN models learn the relationship ${\bf y} \rightarrow {\bf \theta}$, where ${\bf y}$ is a time series response and ${\bf \theta}$ are the parameters of the descriptive model.

Therefore, we require a training set of the form $({\bf y, \theta})$ to train the ANN models. We will first generate such a training set.

In [1]:
# Imports
from ma2_model import simulate, prior
import numpy as np

In [2]:
# Set up simulation
sim = simulate
true_param = [0.6, 0.2]  # true \theta moving average2

data = simulate(true_param)

n = 10000
train_thetas = np.array(prior(n=n))
train_ts = np.expand_dims(np.array([simulate(p, n=100) for p in train_thetas]), 2)

validation_thetas = np.array(prior(n=10000))
validation_ts = np.expand_dims(np.array([simulate(p, n=100) for p in validation_thetas]), 2)

test_thetas = np.array(prior(n=10000))
test_ts = np.expand_dims(np.array([simulate(p, n=100) for p in validation_thetas]), 2)

abc_trial_thetas = np.array(prior(n=30000))
abc_trial_ts = np.expand_dims(np.array([simulate(p, n=100) for p in abc_trial_thetas]), 2)

In [3]:
# Set up training data in the right format
train_ts = train_ts.transpose((0, 2, 1))
validation_ts = validation_ts.transpose((0, 2, 1))
test_ts = test_ts.transpose((0, 2, 1))
data = data.reshape(1,1,100)

In [4]:
# Set input and output shape for the CNN
input_shape = (100,1)
output_shape = 2

In [5]:
# Set up the search space for inference
dmin = [-2, -1]
dmax = [4, 2]

In [6]:
# Routines to normalize and denormalize data
# Makes training easier
def normalize_data(data, dmin, dmax):
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    return (data - dmin)/(dmax-dmin)

def denormalize_data(data, dmin, dmax):
    dmin = np.array(dmin)
    dmax = np.array(dmax)
    denorm = data * (dmax-dmin) + dmin
    return denorm

In [7]:
normed_thetas = normalize_data(train_thetas, dmin, dmax)
normed_thetas_val = normalize_data(validation_thetas, dmin, dmax)

In [8]:
# Import the ANN models
from sciope.models.cnn_regressor import CNNModel
from sciope.models.dnn_regressor import DNNModel
from sciope.models.pen_regressor import PENModel

In [9]:
# Instantiate the models
model_cnn = CNNModel(input_shape, output_shape)
model_dnn = DNNModel(input_shape, output_shape)
model_pen = PENModel(input_shape, output_shape, pen_nr=10)

In [10]:
history_cnn = model_cnn.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=0, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

In [11]:
history_dnn = model_dnn.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=0, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

In [12]:
history_pen = model_pen.train(train_ts, normed_thetas, batch_size=256, 
                      epochs=500, verbose=0, learning_rate=0.001, 
                      early_stopping_patience=5,
                      validation_inputs=validation_ts, validation_targets=normed_thetas_val)

In [13]:
# Routine to test each ANN architecture on MAE
from sklearn.metrics import mean_absolute_error
def test_model(model):
    pred_test = model.predict(test_ts)
    pred_test = denormalize_data(pred_test, dmin, dmax)
    #mae_test = np.mean(abs(pred_test - samples_test), axis=0)
    mae_test = mean_absolute_error(test_thetas, pred_test)
    
    theta_pred = model.predict(data)
    samples_true = np.asarray(true_param)
    theta_pred = denormalize_data(theta_pred, dmin, dmax)
    mae_true = np.mean(np.abs(theta_pred.ravel() - samples_true), axis=0)
    #mae_true = mean_absolute_error(np.asarray(samples_true).reshape(1,2), np.asarray(theta_pred).reshape(1,2))
    return mae_test, mae_true

In [14]:
# Calculate test metrics
mae_test_cnn, mae_true_cnn = test_model(model_cnn)
mae_test_dnn, mae_true_dnn = test_model(model_dnn)
mae_test_pen, mae_true_pen = test_model(model_pen)

In [15]:
print("CNN test MAE = {}, MAE at true point = {}".format(mae_test_cnn, mae_true_cnn))
print("DNN test MAE = {}, MAE at true point = {}".format(mae_test_dnn, mae_true_dnn))
print("PEN test MAE = {}, MAE at true point = {}".format(mae_test_pen, mae_true_pen))

CNN test MAE = 0.7031869420803443, MAE at true point = 0.1392668604850769
DNN test MAE = 0.7157589187285183, MAE at true point = 0.2387733519077301
PEN test MAE = 0.7589048176817108, MAE at true point = 0.12148187458515167


So here we used the ANN models to directly infer the parameters from given time series. The ANN models can also be used as summary statistics in conjunction with ABC inference. The following Class implements one such summary statistic.

In [16]:
from sciope.utilities.summarystats.summary_base import SummaryBase
from sciope.utilities.housekeeping import sciope_logger as ml

class ANN_Statistics(SummaryBase):
    """
    The thetas predicted by ANN models act as summary statistics
    """

    def __init__(self, mean_trajectories=False, use_logger=False):
        self.name = 'ANN_Statistics'
        super(ANN_Statistics, self).__init__(self.name, mean_trajectories, use_logger)
        if self.use_logger:
            self.logger = ml.SciopeLogger().get_logger()
            self.logger.info("ANN_Statistics summary statistic initialized")

    def compute(self, data):
        """
        Calculate the value(s) of the summary statistic(s)
        
        Parameters
        ----------
        data : [type]
            simulated or data set in the form N x S X T - num data points x num species x num time steps
        
        Returns
        -------
        [type]
            computed statistic value
        
        """
        data_arr = np.array(data)
        assert len(data_arr.shape) == 3, "required input shape is (n_points, n_species, n_timepoints)"

        res = model_cnn.predict(data_arr)
        res = denormalize_data(res, dmin, dmax)

        if self.mean_trajectories:
            res = np.asarray(np.mean(res, axis=0))  # returns a scalar, so we cast it as an array

        if self.use_logger:
            self.logger.info("ANN_Statistics summary statistic: processed data matrix of shape {0} and generated summaries"
                             " of shape {1}".format(data.shape, res.shape))
        return res

In [17]:
# Test our new summary statistic
cnn_stat = ANN_Statistics()
predicted_stat = cnn_stat.compute(data)

In [18]:
print(predicted_stat)
stat_mae = np.mean(np.abs(predicted_stat.ravel() - np.asarray(true_param)), axis=0)
print("MAE upon comparison as a statistic = {}".format(stat_mae))

[[0.36439425 0.24292797]]
MAE upon comparison as a statistic = 0.1392668604850769


Now we are ready to set up ABC inference. Here we use Burstiness as a summary statistic. Auto-covariance is more appropriate for MA(2). We can use our ANN model as well, but it should be trained on far more points in order to be accurate enough for use in inference. This notebook is meant to be quick to execute, and so we have used a small training set for exposition.

In [19]:
# The MA2 simulator function
def ma2_sim(param):
    res = np.expand_dims(np.array([simulate(param, n=100)]), 2)
    res = res.transpose((0, 2, 1))    # reshape to N x S X T
    return res

In [20]:
# Generate observed data
obs_data = []
for i in range(20):
    obs_data.append(ma2_sim(true_param))
obs_data = np.asarray(obs_data)
obs_data = obs_data[:, 0, :, :]

In [21]:
from sciope.inference import abc_inference
from sciope.utilities.priors import uniform_prior
from sciope.utilities.summarystats import burstiness

In [22]:
prior_function = uniform_prior.UniformPrior(dmin, dmax)
bs_stat = burstiness.Burstiness()
abc_instance = abc_inference.ABC(obs_data, ma2_sim, prior_function, epsilon=0.01, 
                                 #summaries_function=cnn_stat.compute, # ensure CNN is accurate enough before use
                                 summaries_function=bs_stat.compute,
                                 summaries_divisor=np.max(abc_trial_thetas, axis=0), use_logger=False)

In [23]:
abc_instance.compute_fixed_mean(chunk_size=2)

In [24]:
abc_results = abc_instance.infer(num_samples=50, batch_size=10, chunk_size=2)

In [25]:
# Calculate MAE
acc_samples = np.asarray(abc_results['accepted_samples'])
abc_mae = np.mean(np.abs(acc_samples - np.asarray(true_param).reshape(1, 2)), axis=0)
abc_mae = np.mean(abc_mae)   # mean of both thetas
print("ABC mean inference error = {}".format(abc_mae))

ABC mean inference error = 1.1916615337995151
